In [62]:
import pandas as pd
import numpy as np
import itertools

In [2]:
!ls tmp/data_Dazai

CKVW6cIgxI_Dazai                 audio_slice.ipynb
CKVW6cIgxI_Dazai.zip             audio_slice_on_colabo.ipynb
CKVW6cIgxI_Dazai_slice           train_Dazai_ORIGINAL.csv
CKVW6cIgxI_Dazai_slice.zip       train_details_Dazai_ORIGINAL.csv


In [3]:
df = pd.read_csv('tmp/data_Dazai/train_Dazai_ORIGINAL.csv')
print(df.shape)
df.head(2)

(74, 6)


,ID,author,title,daisy_folder,audio_path,target
0,1Gckfr5bxxhzvoo,CKVW6cIgxI,96bUO3HpHJ,OyDBhKtUqP,train/CKVW6cIgxI/96bUO3HpHJ/OyDBhKtUqP/JbJJRPU...,これは、もう、疑いをいれない。よほどのものである。こと、怠惰に関してだけは、私は、ほんもので...
1,1ti0Fttao7kYi0r,CKVW6cIgxI,diO56wJe9v,Ttv6lyaoDd,train/CKVW6cIgxI/diO56wJe9v/Ttv6lyaoDd/Af6sX3m...,小説修業に就いて語れ。という出題は、私を困惑させた。就職試験を受けにいって、小学校の算術の問...


In [4]:
# 文字数
df['target_length'] = [len(s) for s in df['target']]

In [15]:
thresh = df['target_length'].quantile([(i+1)/15 for i in range(14)])
print(len(thresh))
thresh

14


0.066667      436.466667
0.133333      562.600000
0.200000      722.200000
0.266667     1118.800000
0.333333     1397.000000
0.400000     1902.600000
0.466667     2381.266667
0.533333     3667.333333
0.600000     4061.600000
0.666667     5523.333333
0.733333     7035.666667
0.800000     9225.200000
0.866667    10599.000000
0.933333    13450.333333
Name: target_length, dtype: float64

In [30]:
category = pd.Series([1 for _ in range(df.shape[0])])
for t in thresh:
    category += (df['target_length']>=t)*1
category.value_counts()

13    5
6     5
12    5
4     5
14    5
11    5
1     5
10    5
15    5
5     5
3     5
7     5
9     5
2     5
8     4
dtype: int64

In [31]:
df['target_length_category'] = category

In [49]:
df = df.sort_values('target_length_category')

In [50]:
df['target_length_category'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [57]:
fold = []
for i in df['target_length_category'].unique():
    tmp = df[df['target_length_category']==i]
    fold.append(np.random.choice([i for i in range(tmp.shape[0])], size=tmp.shape[0], replace=False).tolist())

In [64]:
fold = list(itertools.chain.from_iterable(fold))
len(fold)

74

In [65]:
df['fold'] = fold

In [67]:
df['fold'].value_counts()

3    15
2    15
1    15
0    15
4    14
Name: fold, dtype: int64

In [68]:
pd.crosstab(df['fold'], df['target_length_category'])

target_length_category,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
fold,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1


In [70]:
df.columns

Index(['ID', 'author', 'title', 'daisy_folder', 'audio_path', 'target',
       'target_length', 'target_length_category', 'fold'],
      dtype='object')

In [75]:
df_info = df[['ID', 'title', 'fold']]
df_info.shape

(74, 3)

In [71]:
df_detail = pd.read_csv('tmp/data_Dazai/train_details_Dazai_ORIGINAL.csv')
print(df_detail.shape)
df_detail.head()

(22100, 7)


,DETAIL_ID,ID,slice_id,duration,start_time,end_time,target_slice
0,1Gckfr5bxxhzvoo_1,1Gckfr5bxxhzvoo,1,3160.816,0.000,3160.816,これは、もう、疑いをいれない。
1,1Gckfr5bxxhzvoo_2,1Gckfr5bxxhzvoo,2,2847.347,3160.816,6008.163,よほどのものである。
2,1Gckfr5bxxhzvoo_3,1Gckfr5bxxhzvoo,3,2560.000,6008.163,8568.163,こと、怠惰に関してだけは、
3,1Gckfr5bxxhzvoo_4,1Gckfr5bxxhzvoo,4,3030.204,8568.163,11598.367,私は、ほんものである。
4,1Gckfr5bxxhzvoo_5,1Gckfr5bxxhzvoo,5,4048.980,11598.367,15647.347,まさか、それを自慢しているわけではない。


In [92]:
df_all = pd.merge(df_detail, df_info, on='ID', how='left')
df_all.shape

(22100, 9)

In [93]:
df_all.head()

,DETAIL_ID,ID,slice_id,duration,start_time,end_time,target_slice,title,fold
0,1Gckfr5bxxhzvoo_1,1Gckfr5bxxhzvoo,1,3160.816,0.000,3160.816,これは、もう、疑いをいれない。,96bUO3HpHJ,3
1,1Gckfr5bxxhzvoo_2,1Gckfr5bxxhzvoo,2,2847.347,3160.816,6008.163,よほどのものである。,96bUO3HpHJ,3
2,1Gckfr5bxxhzvoo_3,1Gckfr5bxxhzvoo,3,2560.000,6008.163,8568.163,こと、怠惰に関してだけは、,96bUO3HpHJ,3
3,1Gckfr5bxxhzvoo_4,1Gckfr5bxxhzvoo,4,3030.204,8568.163,11598.367,私は、ほんものである。,96bUO3HpHJ,3
4,1Gckfr5bxxhzvoo_5,1Gckfr5bxxhzvoo,5,4048.980,11598.367,15647.347,まさか、それを自慢しているわけではない。,96bUO3HpHJ,3


In [94]:
df_all['fold'].value_counts()

3    5013
2    4716
0    4533
4    3930
1    3908
Name: fold, dtype: int64

In [95]:
df_all['audio'] = [i+'.mp3' for i in df_all['DETAIL_ID']]

In [96]:
df_all.columns

Index(['DETAIL_ID', 'ID', 'slice_id', 'duration', 'start_time', 'end_time',
       'target_slice', 'title', 'fold', 'audio'],
      dtype='object')

In [97]:
df_all = df_all[['audio', 'ID', 'duration', 'target_slice', 'fold']]

In [98]:
df_all = df_all.rename(columns={'target_slice':'target'})
df_all.shape

(22100, 5)

In [100]:
df_all.head()

,audio,ID,duration,target,fold
0,1Gckfr5bxxhzvoo_1.mp3,1Gckfr5bxxhzvoo,3160.816,これは、もう、疑いをいれない。,3
1,1Gckfr5bxxhzvoo_2.mp3,1Gckfr5bxxhzvoo,2847.347,よほどのものである。,3
2,1Gckfr5bxxhzvoo_3.mp3,1Gckfr5bxxhzvoo,2560.000,こと、怠惰に関してだけは、,3
3,1Gckfr5bxxhzvoo_4.mp3,1Gckfr5bxxhzvoo,3030.204,私は、ほんものである。,3
4,1Gckfr5bxxhzvoo_5.mp3,1Gckfr5bxxhzvoo,4048.980,まさか、それを自慢しているわけではない。,3


In [101]:
df_test = df_all[df_all['fold']==4]
df_train = df_all[df_all['fold']!=4]
df_train.shape, df_test.shape

((18170, 5), (3930, 5))

In [102]:
df_train.shape[0] + df_test.shape[0]

22100

In [103]:
df_train.to_csv('audio_Dazai/train.csv', index=False)
df_test.to_csv('audio_Dazai/test.csv', index=False)